In [205]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
FraudFile="data\Fraud\Fraud_Data.csv"
IPFile="data\Fraud\IpAddress_to_Country.csv"
fraud=pd.read_csv(FraudFile,header=0,parse_dates=[1,2])
ip=pd.read_csv(IPFile,header=0)#,parse_dates=[5,6])


In [206]:
fraud.head()
ip.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [207]:
fraud.describe()
len(fraud)

151112

In [208]:
#get country info based on IP
country=[]
#for i in range(50):
for i in range(len(fraud)):
    record_ip=fraud.loc[i,"ip_address"]
    #print(record_ip)
    ip_country=ip[(ip["lower_bound_ip_address"]<=record_ip) & (ip["upper_bound_ip_address"]>=record_ip)]
    if(len(ip_country)==1):
    
        country.append(ip_country['country'].values[0])
    
    else:
        country.append("NA")
    
    
    
    #print(type(ip_country))
    #print(len(ip_country))
    #print(ip_country['country'].values[0])
    #ip_country
    #country.append(ip_country.values[0])
fraud['country']=country

In [209]:
fraud.groupby("country").size()
#len(fraud[fraud["country"]=="NA"])#21966
#len(fraud)#151112

country
Afghanistan                            6
Albania                               13
Algeria                              122
Angola                                24
Antigua and Barbuda                    3
Argentina                            661
Armenia                               15
Australia                           1844
Austria                              435
Azerbaijan                            23
Bahamas                                4
Bahrain                               14
Bangladesh                            37
Barbados                               9
Belarus                               72
Belgium                              409
Belize                                 7
Benin                                  2
Bermuda                                2
Bhutan                                 2
Bolivia                               53
Bonaire; Sint Eustatius; Saba          1
Bosnia and Herzegowina                30
Botswana                               3
Brazil  

In [210]:
#feature engineering 
fraud.head()
time_diff=(fraud['purchase_time']-fraud['signup_time'])
time_diff=time_diff.apply(lambda x:x.seconds)
time_diff
fraud['time_diff']=time_diff


In [212]:
fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,time_diff
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,13882
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA,60085
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,41461


In [213]:
fraud.groupby("user_id").size().max()#confirm user id is unique
fraud.groupby("device_id").size().max() #confirm device id is not unique
fraud.groupby("ip_address").size().max() #confirm ip address is not unique

20

In [214]:
#calculate user count per device 
device_cnt=fraud.groupby('device_id').size().reset_index()
device_cnt=device_cnt.rename(columns={0:"user_per_device_count"})#column name rename
 
fraud = fraud.merge(device_cnt, how='left', on='device_id')
fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,time_diff,user_per_device_count
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,13882,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,1,12
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA,60085,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,41461,1


In [215]:
ip_count=fraud.groupby("ip_address").size().reset_index()
ip_count
ip_count=ip_count.rename(columns={0:"user_per_ip_count"})
fraud=fraud.merge(ip_count,how="left",on="ip_address")
fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,time_diff,user_per_device_count,user_per_ip_count
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,13882,1,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944,1,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,1,12,12
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA,60085,1,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,41461,1,1


In [216]:
#more feature engineer 
#time_diff.apply(lambda x:x.seconds)
fraud["signup_dayofweek"]=fraud["signup_time"].apply(lambda x:x.dayofweek)#Python starts with 0 = Monday.
fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,time_diff,user_per_device_count,user_per_ip_count,signup_dayofweek
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,13882,1,1,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944,1,1,6
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,1,12,12,3
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA,60085,1,1,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,41461,1,1,1


In [217]:
#list(fraud.columns.values)
list(fraud)

['user_id',
 'signup_time',
 'purchase_time',
 'purchase_value',
 'device_id',
 'source',
 'browser',
 'sex',
 'age',
 'ip_address',
 'class',
 'country',
 'time_diff',
 'user_per_device_count',
 'user_per_ip_count',
 'signup_dayofweek']

In [218]:
#feature pick 
columns=[#'user_id',
 #'signup_time',
 #'purchase_time',
 'purchase_value',
 #'device_id',
 'source',
 'browser',
 'sex',
 'age',
 #'ip_address',
 'class',
 'country',
 #'Purchase_since_Signup_gap',
 'time_diff',
 #'count_x',
 #'count_y',
 'user_per_device_count',
 'user_per_ip_count',
 'signup_dayofweek']
fraud=fraud[columns]


In [219]:
# Initialize H2O cluster
# trying with random forest in H2O
#h2o.init()
#h2o.remove_all()

#rest scripts see reference here https://github.com/JifuZhao/DS-Take-Home/blob/master/04.%20Identifying%20Fraudulent%20Activities.ipynb 

In [220]:
'''
#re-sampling data
#here re-sampling is applied to overall dataset. shall we just limit re-sampling to training data only
#technique 1: under-sampling 
fraud_no_count,fraud_yes_count=fraud["class"].value_counts()
#print(fraud["class"].value_counts())
#print(fraud_yes_count)
#print(fraud_no_count)

fraud_yes=fraud[fraud["class"]==1]
fraud_no=fraud[fraud["class"]==0]
fraud_no_under=fraud_no.sample(fraud_yes_count)
#print(fraud_no_under["class"].value_counts())
fraud_new=fraud_yes.append(fraud_no_under)
fraud_new.shape
'''

'\n#re-sampling data\n#here re-sampling is applied to overall dataset. shall we just limit re-sampling to training data only\n#technique 1: under-sampling \nfraud_no_count,fraud_yes_count=fraud["class"].value_counts()\n#print(fraud["class"].value_counts())\n#print(fraud_yes_count)\n#print(fraud_no_count)\n\nfraud_yes=fraud[fraud["class"]==1]\nfraud_no=fraud[fraud["class"]==0]\nfraud_no_under=fraud_no.sample(fraud_yes_count)\n#print(fraud_no_under["class"].value_counts())\nfraud_new=fraud_yes.append(fraud_no_under)\nfraud_new.shape\n'

In [221]:
'''
#re-sampling data
#technique 2: over-sampling
fraud_no_count,fraud_yes_count=fraud["class"].value_counts()

fraud_yes=fraud[fraud["class"]==1]
fraud_no=fraud[fraud["class"]==0]
fraud_yes_over=fraud_yes.sample(fraud_no_count,replace=True)
#print(fraud_no_under["class"].value_counts())
fraud_new=fraud_yes_over.append(fraud_no)
fraud_new.shape
fraud=fraud_new
'''

'\n#re-sampling data\n#technique 2: over-sampling\nfraud_no_count,fraud_yes_count=fraud["class"].value_counts()\n\nfraud_yes=fraud[fraud["class"]==1]\nfraud_no=fraud[fraud["class"]==0]\nfraud_yes_over=fraud_yes.sample(fraud_no_count,replace=True)\n#print(fraud_no_under["class"].value_counts())\nfraud_new=fraud_yes_over.append(fraud_no)\nfraud_new.shape\nfraud=fraud_new\n'

In [222]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

training=fraud[['purchase_value','age','time_diff','user_per_device_count','user_per_ip_count','signup_dayofweek']]
training['source']=le.fit_transform(fraud['source'])
training['browser']=le.fit_transform(fraud['browser'])
training['sex']=le.fit_transform(fraud['sex'])
training['country']=le.fit_transform(fraud['country'])
y=fraud['class']

C:\Users\shanchen\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\shanchen\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\shanchen\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [223]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

clf=DecisionTreeClassifier()
X_train, X_test, y_train, y_test = train_test_split(training, y, test_size=0.7)

print(pd.DataFrame(y_train).groupby("class").size())
 


class
0    41069
1     4264
dtype: int64


In [224]:
#re-sampling data
#technique 2: over-sampling

#get data ready, re-sample the training set only
training_all=X_train
training_all['class']=y_train
training_all.head()

#validation on training_all distribution over class before re-sampling
print(training_all.groupby('class').size())

training_no_cnt,training_yes_cnt=training_all['class'].value_counts()
#split based on label
fraud_yes=training_all[training_all["class"]==1]
fraud_no=training_all[training_all["class"]==0]
#re-sampling
fraud_yes_over=fraud_yes.sample(fraud_no_count,replace=True)
#combine dataset
training_all=fraud_yes_over.append(fraud_no)

#split data into x_train and y_train
X_train=training_all.drop(["class"] ,axis=1)
y_train=training_all["class"]
#verify re-sampling is working
print(training_all.groupby("class").size())
 
    

class
0    41069
1     4264
dtype: int64
class
0    41069
1    41236
dtype: int64


C:\Users\shanchen\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [225]:
clf=clf.fit(X=X_train,y=y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.05%


In [226]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)

Confusion matrix:
 [[91544  4348]
 [ 4064  5823]]


In [227]:
#calculate feature importance 
importance=clf.feature_importances_
features=list(X_train.columns)

importances = sorted(zip(features, importance), key=lambda x:x[1], reverse=True)
for feature, val in importances:
    print('{0:10s} | {1:.5f}'.format(feature, val))

user_per_device_count | 0.44899
time_diff  | 0.20758
purchase_value | 0.09735
age        | 0.07667
country    | 0.05508
signup_dayofweek | 0.04655
browser    | 0.02741
source     | 0.02422
sex        | 0.01205
user_per_ip_count | 0.00409


In [228]:
#model evaluation
from sklearn.model_selection import cross_val_score
cross_val_score(clf, X=training,y=y, cv=10,scoring='f1_macro')

array([ 0.7674235 ,  0.75102017,  0.75973917,  0.7660937 ,  0.75235354,
        0.75683968,  0.75506462,  0.76482431,  0.75296755,  0.76468514])

In [229]:
#model evaluation
cross_val_score(clf, X=training,y=y, cv=10,scoring='accuracy')

array([ 0.91742209,  0.91059493,  0.91350672,  0.91661703,  0.91158759,
        0.91238171,  0.91152141,  0.91456555,  0.91072728,  0.91569056])

In [230]:
cross_val_score(clf, X=training,y=y, cv=10,scoring='precision')

array([ 0.54166667,  0.52432778,  0.529375  ,  0.54368322,  0.52528445,
        0.53104787,  0.5316947 ,  0.54522454,  0.51881555,  0.53375   ])

In [231]:
cross_val_score(clf, X=training,y=y, cv=10,scoring='recall')

array([ 0.60522599,  0.58303887,  0.59929329,  0.61484099,  0.58727915,
        0.57597173,  0.57526502,  0.60989399,  0.59081272,  0.604947  ])

In [232]:
result=pd.DataFrame(np.array(y_test))
result['Prediction']=predictions
result=result.rename(columns={0:"Actual"})

In [233]:
result.head()
result.groupby(['Actual','Prediction']).size()

Actual  Prediction
0       0             85672
        1             10220
1       0              8861
        1              1026
dtype: int64

In [234]:
#summary
#how soon user is making a purchase is an important signal to detect fraudulent activities
#also since data is imbalanced. I will try with different sampling strategy to improve it.  #tried, resampling does not help for precision and recall here
#if I have more time, I will spend more time in feature engineering